Detector de ejercicios de press de banca

In [99]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Inicializa MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8)


def calculate_angle(a, b, c):
    """Calcula el ángulo entre tres puntos clave."""
    a = np.array([a.x, a.y])  # Primer punto
    b = np.array([b.x, b.y])  # Punto central (vértice)
    c = np.array([c.x, c.y])  # Tercer punto

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

class PressBancaCounter:
    def __init__(self):
        self.reps = 0
        self.state = "up"  # Estado inicial: en extensión completa

    def update(self, left_angle, right_angle):
        # Umbrales para los ángulos
        min_flex_angle = 80  # Punto máximo de flexión
        max_extend_angle = 100  # Punto máximo de extensión

        # Determinar el estado actual basado en los ángulos
        if left_angle >= max_extend_angle and right_angle >= max_extend_angle:
            if self.state == "down":
                self.reps += 1  # Contar repetición cuando se vuelva a extensión
                
            self.state = "up"
        elif left_angle <= min_flex_angle and right_angle <= min_flex_angle:
            self.state = "down"

        return self.reps
class Rm:
    def __init__(self, weight):
        self.rm = 0.0
        self.weight = weight   
           
    def Epley(self, reps):
        newReps = self.RepsToInt(reps)
        self.rm = (self.weight*newReps*0.033)+self.weight
        return self.rm
    
    def Brzycki(self, reps):
        newReps = self.RepsToInt(reps)
        self.rm = self.weight/(1.0278 - (0.0278*newReps))
        return self.rm
    
    def EpleyReducida(self, reps):
        newReps = self.RepsToInt(reps)
        work = 0.03
        suma = 0
        for i in range(5):
            suma += (self.weight*newReps*work)+self.weight
            work -= 0.001
        suma = suma/5
        self.rm = round(suma, 1)
        return self.rm
    
    def RmVelocidad(self, reps, velocidades):
        newReps = self.RepsToInt(reps)
        suma = 0
        
        for velocidad in velocidades:
            suma += self.weight / (1 - (velocidad * 0.03))
        suma = suma/len(velocidades)
        self.rm = round(suma,1)
        return self.rm    
        """#hacer el cambio de la formula
        VR = max(velocidad / 0.278, 0.1)
        suma += (self.weight * (1 + 0.0333 * newReps)) * (1 / VR)
    suma = suma/len(velocidades)
    self.rm = round(suma,1)
    return self.rm"""
    
    def RepsToInt(self, reps):
        return float(reps.split(':')[-1].strip())
    
class VelocityCalculator:
    def __init__(self):
        self.last_time = None
        self.velocities = []
        self.fps_original = 0
        self.actual_fps = 0
        self.frame_high = 0
        # Nuevos atributos para tiempo transcurrido
        self.start_time = None  # Tiempo en posición baja
        self.end_time = None  # Tiempo en posición alta
        self.times = []  # Lista para almacenar los tiempos transcurridos entre eventos
        self.distancia = 0.47

    def update(self, tiempo_estimado):
        """
        Calcula y almacena la velocidad basada en el tiempo entre eventos.
        """
        if tiempo_estimado > 0:  # Prevenir divisiones por cero
            velocidad = 1 / tiempo_estimado  # Velocidad proporcional al tiempo
            velocidad_ajustada = velocidad * (self.fps_original / self.actual_fps)
            self.velocities.append(velocidad)
            return velocidad_ajustada
            #esto es para el caso de utilizar una medida de referencia (longitud del brazo)
            """velocidad = self.distancia / tiempo_estimado 
            self.velocities.append(velocidad)
            return velocidad"""

        #return None

    def start_timing(self, wrist_position_y):
        """Inicia el temporizador y registra la posición de la muñeca en el punto bajo."""
        self.start_time = time.time()
        self.start_position_y = wrist_position_y * self.frame_high
        
    def stop_timing(self, wrist_position_y):
        """Detiene el temporizador y registra la posición de la muñeca en el punto alto."""
        self.end_time = time.time()
        if self.start_time is not None and self.end_time is not None:
            elapsed_time = self.end_time - self.start_time

            
            corrected_elapsed_time = (elapsed_time * self.actual_fps) / self.fps_original
            # Almacena el tiempo corregido para análisis posterior
            self.times.append(corrected_elapsed_time)

            self.start_time = None  # Reinicia para la próxima repetición
            return corrected_elapsed_time

        return None, None, None

    def get_average_time(self):
        """Calcula el tiempo promedio entre eventos registrados."""
        if self.times:
            return np.mean(self.times)
        return 0.0

    def get_average_velocity(self):
        """Calcula la velocidad media de todas las repeticiones registradas."""
        if self.velocities:
            return np.mean(self.velocities)
        return 0.0

    def set_actual_fps(self, actual_fps):
        """Establece el FPS actual, útil para ajustar las velocidades cuando se ralentiza el video."""
        self.actual_fps = actual_fps

class PressBancaState: 
    def __init__(self):
        self.puntuacion_extension = 0.0
        self.puntuacion_bajo = 0.0
        self.puntuacion_rom = 0.0
        self.repeticion = 0
        self.state = "up"  # Estado inicial (extensión máxima)
        self.tiempo_estimado = 0
        self.ROM = []

    def calculate_extension_score(self, left_angle, right_angle):
        """Calcula la puntuación de la extensión en el punto máximo."""
        def score_angle(angle):
            if 150 <= angle <= 160:
                return 9 + (angle - 150) / 10  # Escala 9-10
            elif 130 <= angle < 150:
                return 7 + (angle - 130) / 20 * 2  # Escala 7-8
            elif 110 <= angle < 130:
                return 4 + (angle - 110) / 20 * 2  # Escala 4-6
            elif angle < 110:
                return max(0, (angle - 100) / 10 * 3)  # Escala 0-3
            return 0

        return (score_angle(left_angle) + score_angle(right_angle)) / 2

    def calculate_contraction_score(self, left_angle, right_angle):
        """Calcula la puntuación de la contracción en el punto máximo."""
        def score_angle(angle):
            if 30 <= angle <= 40:
                return 9 + (40 - angle) / 10  # Escala 9-10
            elif 45 <= angle <= 55:
                return 7 + (55 - angle) / 10 * 2  # Escala 7-8
            elif 60 <= angle <= 70:
                return 4 + (70 - angle) / 10 * 2  # Escala 4-6
            elif angle > 75:
                return max(0, 3 - (angle - 75) / 10 * 3)  # Escala 0-3
            return 0

        return (score_angle(left_angle) + score_angle(right_angle)) / 2

    def detect_press_banca(self, landmarks, counter, velocity):
        if landmarks:
            # Puntos clave relevantes
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
            
            # Ángulos de codos
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            
            # Detectar estado y calcular puntuaciones según el estado
            if self.state == "up" and left_elbow_angle <= 70 and right_elbow_angle <= 70:
                # Punto máximo de flexión (contracción)
                self.puntuacion_bajo = self.calculate_contraction_score(left_elbow_angle, right_elbow_angle)
                self.state = "down"
                velocity.start_timing(left_wrist.y)  # Inicia el temporizador

            elif self.state == "down" and left_elbow_angle >= 135 and right_elbow_angle >= 135:
                # Punto máximo de extensión
                self.puntuacion_extension = self.calculate_extension_score(left_elbow_angle, right_elbow_angle)
                self.state = "up"
                self.tiempo_estimado = velocity.stop_timing(left_wrist.y)  # Finalizar tiempo y posición
                # Calcular ROM solo al finalizar una repetición completa
                velocity.update(self.tiempo_estimado)
                
                self.puntuacion_rom = np.mean([self.puntuacion_extension, self.puntuacion_bajo])
                self.ROM.append(self.puntuacion_rom)
            # Actualizar repetición al volver a la extensión
            last_move = counter.state
            self.repeticion = counter.update(left_elbow_angle, right_elbow_angle)
        # Retornar siempre los valores actuales
        return [
            f"Extension: {self.puntuacion_extension:.2f}",
            f"Contraccion: {self.puntuacion_bajo:.2f}",
            f"ROM: {self.puntuacion_rom:.2f}",
            f"Repeticiones: {self.repeticion}",
            f"velocidad: {velocity.get_average_velocity():.2f} m/s"
        ]
            
def start_press(peso, video_path):
    # Cambia el argumento a la ruta de tu video
    cap = cv2.VideoCapture(video_path)
    counter = PressBancaCounter()
    velocity = VelocityCalculator()
    state = PressBancaState()
    RM = Rm(float(peso))

    prev_time = None
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Fin del video o archivo no encontrado.")
            
            break
        
        velocity.frame_high = frame.shape[0]
        # Convierte la imagen a RGB (MediaPipe requiere RGB)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb_frame.flags.writeable = False

        # Procesa la detección de poses
        results = pose.process(rgb_frame)

        # Visualiza los resultados
        rgb_frame.flags.writeable = True
        frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)
        
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
            )

            # Detecta el ejercicio basado en las posiciones
            velocity.fps_original = (cap.get(cv2.CAP_PROP_FPS))
            
            current_time = time.time()
            if prev_time is not None:
                # Calcula el tiempo entre fotogramas
                delta_time = current_time - prev_time
                # Puedes calcular FPS reales del procesamiento
                velocity.actual_fps = 1 / delta_time
            prev_time = current_time
            
            exercise1, exercise2, exercise3, exercise4, exercise5 = state.detect_press_banca(results.pose_landmarks.landmark, counter, velocity)

            frame_count += 1
            # Ajuste dinámico del tamaño del texto según el tamaño del frame
            height, width, _ = frame.shape
            font_scale = width / 950  # Escala el tamaño del texto en función del ancho del video
            font = cv2.FONT_HERSHEY_SIMPLEX

            # Ponemos el texto en la esquina superior izquierda
            cv2.putText(
                frame, exercise1, (10, 40), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
            )
            cv2.putText(
                frame, exercise2, (10, 80), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
            )
            cv2.putText(
                frame, exercise3, (10, 120), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
            )
            cv2.putText(
                frame, exercise4, (10, 160), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
            )
            cv2.putText(
                frame, exercise5, (10, 200), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
            )

        # Muestra la salida
        cv2.imshow('Gym Exercise Analysis', frame)

        # Salir con 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    Epley = RM.Epley(exercise4)
    Brzycki  = RM.Brzycki(exercise4)
    EpleyReducido = RM.EpleyReducida(exercise4)
    rmVelocidad = RM.RmVelocidad(exercise4, velocity.velocities)
    reps = RM.RepsToInt(exercise4)
    ROM = state.ROM
    cap.release()
    cv2.destroyAllWindows()
    return f"repeticiones totales: {reps:.2f},\n RM con EPLEY: {Epley:.2f},\n RM con BRZYCKI(bueno a repeticiones bajas): {Brzycki:.2f},\n RM con EPLEYREDUCIDO: {EpleyReducido:.2f},\n RM con Velocidad: {rmVelocidad:.2f},\n  Velocidad media: {velocity.get_average_velocity():.2f} m/s"


Detector de ejercicios de peso muerto

In [100]:
import cv2
import mediapipe as mp
import numpy as np
import time

class Rms:
    def __init__(self, weight):
        self.rm = 0.0
        self.weight = weight   
           
    def Epley(self, reps):
        self.rm = (self.weight*reps*0.033)+self.weight
        return self.rm
    
    def Brzycki(self, reps):
        self.rm = self.weight/(1.0278 - (0.0278*reps))
        return self.rm
    
    def EpleyReducida(self, reps):
        work = 0.03
        suma = 0
        for i in range(5):
            suma += (self.weight*reps*work)+self.weight
            work -= 0.001
        suma = suma/5
        self.rm = round(suma, 1)
        return self.rm
    
    def RmVelocidad(self, reps, velocidades):
        suma = 0
        for velocidad in velocidades:
            suma += self.weight / (1 - (velocidad * 0.03))
        suma = suma/len(velocidades)
        self.rm = round(suma,1)
        return self.rm    



class VelocityCalculators:
    def __init__(self):
        self.last_time = None
        self.velocities = []
        self.fps_original = 0
        self.actual_fps = 0
        self.frame_high = 0
        # Nuevos atributos para tiempo transcurrido
        self.start_time = None  # Tiempo en posición baja
        self.end_time = None  # Tiempo en posición alta
        self.times = []  # Lista para almacenar los tiempos transcurridos entre eventos
        self.velocidad = 0

    def update(self, tiempo_estimado):
        """
        Calcula y almacena la velocidad basada en el tiempo entre eventos.
        """
        if tiempo_estimado > 0:  # Prevenir divisiones por cero
            self.velocidad = 1 / tiempo_estimado  # Velocidad proporcional al tiempo
            self.velocidad = self.velocidad * (self.actual_fps / self.fps_original)
            self.velocities.append(self.velocidad)
            return self.velocidad

        #return None

    def start_timing(self, wrist_position_y):
        """Inicia el temporizador y registra la posición de la muñeca en el punto bajo."""
        self.start_time = time.time()
        self.start_position_y = wrist_position_y * self.frame_high
        
    def stop_timing(self, wrist_position_y):
        """Detiene el temporizador y registra la posición de la muñeca en el punto alto."""
        self.end_time = time.time()
        if self.start_time is not None and self.end_time is not None:
            elapsed_time = self.end_time - self.start_time

            
            corrected_elapsed_time = (elapsed_time * self.actual_fps) / self.fps_original
            # Almacena el tiempo corregido para análisis posterior
            self.times.append(corrected_elapsed_time)

            self.start_time = None  # Reinicia para la próxima repetición
            return corrected_elapsed_time

        return None

    def get_average_time(self):
        """Calcula el tiempo promedio entre eventos registrados."""
        if self.times:
            return np.mean(self.times)
        return 0.0

    def get_average_velocity(self):
        """Calcula la velocidad media de todas las repeticiones registradas."""
        if self.velocities:
            return np.mean(self.velocities)
        return 0.0

    def set_actual_fps(self, actual_fps):
        """Establece el FPS actual, útil para ajustar las velocidades cuando se ralentiza el video."""
        self.actual_fps = actual_fps

class DeadliftAnalyzer:
    def __init__(self):
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose()
        self.reps = 0
        self.direction = None  # 'up' or 'down'
        self.rom = 0
        self.ROM = []
        self.puntuacion_flexion = 0
        self.puntuacion_extension = 0
        self.tiempo_estimado = 0

    def calculate_angle(self, a, b, c):
        a = np.array(a)  # First point
        b = np.array(b)  # Mid point
        c = np.array(c)  # End point

        radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
        angle = np.abs(radians * 180.0 / np.pi)

        if angle > 180.0:
            angle = 360 - angle

        return angle
    
    def calculate_extension_score(self, back_angle, knee_angle):
        """Calcula la puntuación de la extensión en el punto máximo considerando espalda y rodillas."""
        def score_angle(angle, min_good, max_good):
            """Pondera un ángulo con una escala específica."""
            if min_good <= angle <= max_good:
                return 9 + (angle - min_good) / (max_good - min_good)  # Escala 9-10
            elif (min_good - 20) <= angle < min_good:
                return 7 + (angle - (min_good - 20)) / 20 * 2  # Escala 7-8
            elif (min_good - 40) <= angle < (min_good - 20):
                return 4 + (angle - (min_good - 40)) / 20 * 2  # Escala 4-6
            elif angle < (min_good - 40):
                return max(0, (angle - (min_good - 50)) / 10 * 3)  # Escala 0-3
            return 0

        back_score = score_angle(back_angle, 170, 180)  # Escala de extensión para espalda
        knee_score = score_angle(knee_angle, 170, 180)  # Escala de extensión para rodillas

        # Ponderación igual para espalda y rodillas
        return (back_score + knee_score) / 2


    def calculate_contraction_score(self, back_angle, knee_angle):
        """
        Calcula la puntuación de la contracción en el punto máximo considerando espalda y rodillas.
        """
        def score_angle(angle, min_good, max_good):
            """Pondera un ángulo con una escala específica."""
            if min_good <= angle <= max_good:
                return 9 + (max_good - angle) / (max_good - min_good)  # Escala 9-10
            elif (max_good + 10) <= angle <= (max_good + 20):
                return 7 + (max_good + 20 - angle) / 10 * 2  # Escala 7-8
            elif (max_good + 20) <= angle <= (max_good + 30):
                return 4 + (max_good + 30 - angle) / 10 * 2  # Escala 4-6
            elif angle > (max_good + 30):
                return max(0, 3 - (angle - (max_good + 30)) / 10 * 3)  # Escala 0-3
            return 0

        back_score = score_angle(back_angle, 35, 50)  # Escala de contracción para espalda
        knee_score = score_angle(knee_angle, 65, 75)  # Escala de contracción para rodillas

        # Ponderación igual para espalda y rodillas
        return (back_score + knee_score) / 2


    def deadlift_detector(self, image, view, velocity): 
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = self.pose.process(image_rgb)

        if not results.pose_landmarks:
            return image

        landmarks = results.pose_landmarks.landmark
        height, width, _ = image.shape

        # Coordinates for key points
        hip = [landmarks[self.mp_pose.PoseLandmark.RIGHT_HIP.value].x * width,
            landmarks[self.mp_pose.PoseLandmark.RIGHT_HIP.value].y * height]
        knee = [landmarks[self.mp_pose.PoseLandmark.RIGHT_KNEE.value].x * width,
                landmarks[self.mp_pose.PoseLandmark.RIGHT_KNEE.value].y * height]
        ankle = [landmarks[self.mp_pose.PoseLandmark.RIGHT_ANKLE.value].x * width,
                landmarks[self.mp_pose.PoseLandmark.RIGHT_ANKLE.value].y * height]

        shoulder = [landmarks[self.mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * width,
                    landmarks[self.mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * height]
        left_wrist = [landmarks[self.mp_pose.PoseLandmark.LEFT_WRIST.value].y*height]

        if view == 'side':
            back_angle = self.calculate_angle(shoulder, hip, knee)
            knee_angle = self.calculate_angle(hip, knee, ankle)
        elif view == 'front':
            left_hip = [landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].x * width,
                        landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].y * height]
            right_hip = [landmarks[self.mp_pose.PoseLandmark.RIGHT_HIP.value].x * width,
                        landmarks[self.mp_pose.PoseLandmark.RIGHT_HIP.value].y * height]
            left_knee = [landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].x * width,
                        landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].y * height]
            right_knee = [landmarks[self.mp_pose.PoseLandmark.RIGHT_KNEE.value].x * width,
                        landmarks[self.mp_pose.PoseLandmark.RIGHT_KNEE.value].y * height]
            left_ankle = [landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].x * width,
                        landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].y * height]
            right_ankle = [landmarks[self.mp_pose.PoseLandmark.RIGHT_ANKLE.value].x * width,
                        landmarks[self.mp_pose.PoseLandmark.RIGHT_ANKLE.value].y * height]

            back_angle = self.calculate_angle(left_hip, shoulder, right_hip)
            knee_angle = (self.calculate_angle(left_hip, left_knee, left_ankle) +
                        self.calculate_angle(right_hip, right_knee, right_ankle)) / 2
        else:
            raise ValueError("Invalid view type. Choose 'side' or 'front'.")

        # Rep counting logic
        if back_angle > 150 and knee_angle > 150:
            if self.direction == 'down':  # Completa la repetición
                self.reps += 1
                self.puntuacion_extension = self.calculate_extension_score(back_angle, knee_angle)
                self.rom = np.mean([self.puntuacion_extension , self.puntuacion_flexion])
                self.ROM.append(self.rom)
                self.tiempo_estimado = velocity.stop_timing(left_wrist)
                if self.tiempo_estimado is not None:
                    velocity.update(self.tiempo_estimado)
            self.direction = 'up'
            
            
        elif back_angle < 95 and knee_angle < 100:
            if self.direction == 'up':  # Empieza la fase de flexión
                self.puntuacion_flexion = self.calculate_contraction_score(back_angle, knee_angle)   
            self.direction = 'down'
            velocity.start_timing(left_wrist)

        # Visualize angles and rep count
        cv2.putText(image, f'Reps: {self.reps:.2f}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(image, f'Extension: {self.puntuacion_extension:.2f}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(image, f'Flexion: {self.puntuacion_flexion:.2f}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(image, f'ROM: {self.rom:.2f}', (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(image, f'velocidad: {velocity.velocidad:.2f} m/s', (10, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        

        # Draw pose landmarks
        self.mp_drawing.draw_landmarks(image, results.pose_landmarks, self.mp_pose.POSE_CONNECTIONS)

        return [image, self.reps]

    def release(self):
        self.pose.close()

def start_pesomuerto(peso, video_path):
    analyzer = DeadliftAnalyzer()
    velocity = VelocityCalculators()
    RM = Rms(float(peso))
    view = "side"
    cap = cv2.VideoCapture(video_path)
    
    prev_time = None
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            print("Fin del video o archivo no encontrado")
            break
        
        velocity.fps_original = (cap.get(cv2.CAP_PROP_FPS))
        
        current_time = time.time()
        if prev_time is not None:
            # Calcula el tiempo entre fotogramas
            delta_time = current_time - prev_time
            # Puedes calcular FPS reales del procesamiento
            velocity.actual_fps = 1 / delta_time
        prev_time = current_time
        
        analyzed_frame, repeticiones = analyzer.deadlift_detector(frame, view, velocity)

        frame_count += 1
        
        
        cv2.imshow('Deadlift Analyzer', analyzed_frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    Epley = RM.Epley(repeticiones)
    Brzycki  = RM.Brzycki(repeticiones)
    EpleyReducido = RM.EpleyReducida(repeticiones)
    rmVelocidad = RM.RmVelocidad(repeticiones, velocity.velocities)
    ROM = analyzer.ROM   
    cap.release()
    cv2.destroyAllWindows()
    analyzer.release()
    return f"repeticiones totales: {repeticiones:.2f},\n RM con EPLEY: {Epley:.2f},\n RM con BRZYCKI(bueno a repeticiones bajas): {Brzycki:.2f},\n RM con EPLEYREDUCIDO: {EpleyReducido:.2f},\n RM con Velocidad: {rmVelocidad:.2f},\n  Velocidad media: {velocity.get_average_velocity():.2f} m/s"

Detector de ejercicio de sentadilla

In [101]:
#SENTADILLAS

import cv2
import mediapipe as mp
import numpy as np
import time

# Inicializa MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8)


def calculate_angle(a, b, c):
    """Calcula el ángulo entre tres puntos clave."""
    a = np.array([a.x, a.y])  # Primer punto
    b = np.array([b.x, b.y])  # Punto central (vértice)
    c = np.array([c.x, c.y])  # Tercer punto

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

class SentadillasCounter:
    def __init__(self):
        self.reps = 0
        self.state = "up"  # Estado inicial: en extensión completa

    def update(self, left_angle, right_angle):
        # Umbrales para los ángulos
        min_flex_angle = 80  # Punto máximo de flexión
        max_extend_angle = 135  # Punto máximo de extensión

        # Detectar transición de "up" a "down"
        if self.state == "up" and left_angle <= min_flex_angle and right_angle <= min_flex_angle:
            self.state = "down"

        # Detectar transición de "down" a "up"
        elif self.state == "down" and left_angle >= max_extend_angle and right_angle >= max_extend_angle:
            self.reps += 1  # Contar una repetición al volver a "up"
            self.state = "up"

        return self.reps
class Rm:
    def __init__(self, weight):
        self.rm = 0.0
        self.weight = weight   
           
    def Epley(self, reps):
        newReps = self.RepsToInt(reps)
        self.rm = (self.weight*newReps*0.033)+self.weight
        return self.rm
    
    def Brzycki(self, reps):
        newReps = self.RepsToInt(reps)
        self.rm = self.weight/(1.0278 - (0.0278*newReps))
        return self.rm
    
    def EpleyReducida(self, reps):
        newReps = self.RepsToInt(reps)
        work = 0.03
        suma = 0
        for i in range(5):
            suma += (self.weight*newReps*work)+self.weight
            work -= 0.001
        suma = suma/5
        self.rm = round(suma, 1)
        return self.rm
    
    def RmVelocidad(self, reps, velocidades):
        newReps = self.RepsToInt(reps)
        suma = 0
        
        for velocidad in velocidades:
            suma += self.weight / (1 - (velocidad * 0.03))
        suma = suma/len(velocidades)
        self.rm = round(suma,1)
        return self.rm    
        """#hacer el cambio de la formula
        VR = max(velocidad / 0.278, 0.1)
        suma += (self.weight * (1 + 0.0333 * newReps)) * (1 / VR)
    suma = suma/len(velocidades)
    self.rm = round(suma,1)
    return self.rm"""
    
    def RepsToInt(self, reps):
        return float(reps.split(':')[-1].strip())
    
class VelocityCalculator:
    def __init__(self):
        self.last_time = None
        self.velocities = []
        self.fps_original = 0
        self.actual_fps = 0
        self.frame_high = 0
        # Nuevos atributos para tiempo transcurrido
        self.start_time = None  # Tiempo en posición baja
        self.end_time = None  # Tiempo en posición alta
        self.times = []  # Lista para almacenar los tiempos transcurridos entre eventos
        self.distancia = 0.47

    def update(self, tiempo_estimado):
        """
        Calcula y almacena la velocidad basada en el tiempo entre eventos.
        """
        if tiempo_estimado > 0:  # Prevenir divisiones por cero
            velocidad = 1 / tiempo_estimado  # Velocidad proporcional al tiempo
            velocidad_ajustada = velocidad * (self.fps_original / self.actual_fps)
            self.velocities.append(velocidad)
            return velocidad_ajustada
            #esto es para el caso de utilizar una medida de referencia (longitud del brazo)
            """velocidad = self.distancia / tiempo_estimado 
            self.velocities.append(velocidad)
            return velocidad"""

        #return None

    def start_timing(self, wrist_position_y):
        """Inicia el temporizador y registra la posición de la muñeca en el punto bajo."""
        self.start_time = time.time()
        self.start_position_y = wrist_position_y * self.frame_high
        
    def stop_timing(self, wrist_position_y):
        """Detiene el temporizador y registra la posición de la muñeca en el punto alto."""
        self.end_time = time.time()
        if self.start_time is not None and self.end_time is not None:
            elapsed_time = self.end_time - self.start_time

            
            corrected_elapsed_time = (elapsed_time * self.actual_fps) / self.fps_original
            # Almacena el tiempo corregido para análisis posterior
            self.times.append(corrected_elapsed_time)

            self.start_time = None  # Reinicia para la próxima repetición
            return corrected_elapsed_time

        return None, None, None

    def get_average_time(self):
        """Calcula el tiempo promedio entre eventos registrados."""
        if self.times:
            return np.mean(self.times)
        return 0.0

    def get_average_velocity(self):
        """Calcula la velocidad media de todas las repeticiones registradas."""
        if self.velocities:
            return np.mean(self.velocities)
        return 0.0

    def set_actual_fps(self, actual_fps):
        """Establece el FPS actual, útil para ajustar las velocidades cuando se ralentiza el video."""
        self.actual_fps = actual_fps

class SentadillasState: 
    def __init__(self):
        self.puntuacion_extension = 0.0
        self.puntuacion_bajo = 0.0
        self.puntuacion_rom = 0.0
        self.repeticion = 0
        self.state = "up"  # Estado inicial (extensión máxima)
        self.tiempo_estimado = 0
        self.ROM = []

    def calculate_extension_score(self, left_angle, right_angle):
        """Calcula la puntuación de la extensión en el punto máximo con ajustes más relajados."""
        def score_angle(angle):
            if 150 <= angle <= 180:
                return 9 + (angle - 150) / 20  # Escala 9-10
            elif 130 <= angle < 150:
                return 7 + (angle - 130) / 20 * 2  # Escala 7-8
            elif 110 <= angle < 130:
                return 4 + (angle - 110) / 20 * 3  # Escala 4-6
            elif angle < 110:
                return max(0, (angle - 90) / 20 * 4)  # Escala 0-3
            return 0

        return (score_angle(left_angle) + score_angle(right_angle)) / 2

    def calculate_contraction_score(self, left_angle, right_angle):
        """Calcula la puntuación de la contracción en el punto máximo con ajustes."""
        def score_angle(angle):
            if 1 <= angle <= 75:
                score = 9 + (75 - angle) / 15
                return min(score, 10)
            elif 75 < angle <= 85:
                return 7 + (85 - angle) / 10 * 2  # Escala 7-8
            elif 85 < angle <= 95:
                return 4 + (95 - angle) / 10 * 3  # Escala 4-6
            elif angle > 95:
                return max(0, 3 - (95 - angle) / 10 * 2)  # Escala 0-3
            return 0
        
        score_izquierda = score_angle(left_angle)
        score_derecha = score_angle(right_angle)

        if abs(score_izquierda - score_derecha) >= 10:
            if score_izquierda < score_derecha:
                return (score_izquierda * 0.75 + score_derecha * 0.25)
            else:
                return (score_izquierda* 0.25 + score_derecha* 0.75)
        
        return (score_angle(left_angle) + score_angle(right_angle)) / 2
        

    def detect_squat(self, landmarks, counter, velocity):
        if landmarks:
            # Puntos clave relevantes
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

            # Ángulos de caderas y rodillas
            left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            
            # Detectar estado y calcular puntuaciones según el estado
            if self.state == "up" and left_hip_angle <= 95 and right_hip_angle <= 95:
                # Punto máximo de flexión (abajo)
                self.puntuacion_bajo = self.calculate_contraction_score(left_hip_angle, right_hip_angle)
                self.state = "down"
                velocity.start_timing(left_ankle.y)  # Inicia el temporizador

            elif self.state == "down" and left_hip_angle >= 135 and right_hip_angle >= 135:
                # Punto máximo de extensión (arriba)
                self.puntuacion_extension = self.calculate_extension_score(left_hip_angle, right_hip_angle)
                self.state = "up"
                self.tiempo_estimado = velocity.stop_timing(left_ankle.y)  # Finalizar tiempo y posición
                velocity.update(self.tiempo_estimado)

                self.puntuacion_rom = np.mean([self.puntuacion_extension, self.puntuacion_bajo])
                self.ROM.append(self.puntuacion_rom)

            # Actualizar repetición al volver a la extensión
            last_move = counter.state
            self.repeticion = counter.update(left_hip_angle, right_hip_angle)

        # Retornar siempre los valores actuales
        return [
            f"Extension: {self.puntuacion_extension:.2f}",
            f"Contraccion: {self.puntuacion_bajo:.2f}",
            f"ROM: {self.puntuacion_rom:.2f}",
            f"Repeticiones: {self.repeticion}",
            f"velocidad: {velocity.get_average_velocity():.2f} m/s"
        ]

            
def start_sentadilla(peso, video_path):
    # Cambia el argumento a la ruta de tu video
    cap = cv2.VideoCapture(video_path)
    counter = SentadillasCounter()
    velocity = VelocityCalculator()
    state = SentadillasState()
    RM = Rm(float(peso))

    prev_time = None
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Fin del video o archivo no encontrado.")
            
            break
        
        velocity.frame_high = frame.shape[0]
        # Convierte la imagen a RGB (MediaPipe requiere RGB)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb_frame.flags.writeable = False

        # Procesa la detección de poses
        results = pose.process(rgb_frame)

        # Visualiza los resultados
        rgb_frame.flags.writeable = True
        frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)
        
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
            )

            # Detecta el ejercicio basado en las posiciones
            velocity.fps_original = (cap.get(cv2.CAP_PROP_FPS))
            
            current_time = time.time()
            if prev_time is not None:
                # Calcula el tiempo entre fotogramas
                delta_time = current_time - prev_time
                # Puedes calcular FPS reales del procesamiento
                velocity.actual_fps = 1 / delta_time
            prev_time = current_time
            
            exercise1, exercise2, exercise3, exercise4, exercise5 = state.detect_squat(results.pose_landmarks.landmark, counter, velocity)

            frame_count += 1
            # Ajuste dinámico del tamaño del texto según el tamaño del frame
            height, width, _ = frame.shape
            font_scale = width / 950  # Escala el tamaño del texto en función del ancho del video
            font = cv2.FONT_HERSHEY_SIMPLEX

            # Ponemos el texto en la esquina superior izquierda
            cv2.putText(
                frame, exercise1, (10, 40), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
            )
            cv2.putText(
                frame, exercise2, (10, 80), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
            )
            cv2.putText(
                frame, exercise3, (10, 120), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
            )
            cv2.putText(
                frame, exercise4, (10, 160), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
            )
            cv2.putText(
                frame, exercise5, (10, 200), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
            )

        # Muestra la salida
        cv2.imshow('Gym Exercise Analysis', frame)

        # Salir con 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    Epley = RM.Epley(exercise4)
    Brzycki  = RM.Brzycki(exercise4)
    EpleyReducido = RM.EpleyReducida(exercise4)
    rmVelocidad = RM.RmVelocidad(exercise4, velocity.velocities)
    reps = RM.RepsToInt(exercise4)
    ROM = state.ROM
    cap.release()
    cv2.destroyAllWindows()
    return f"repeticiones totales: {reps:.2f},\n RM con EPLEY: {Epley:.2f},\n RM con BRZYCKI(bueno a repeticiones bajas): {Brzycki:.2f},\n RM con EPLEYREDUCIDO: {EpleyReducido:.2f},\n RM con Velocidad: {rmVelocidad:.2f},\n  Velocidad media: {velocity.get_average_velocity():.2f} m/s"
    

Interfaz gráfica

In [102]:
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
import cv2
from PIL import Image, ImageTk
import threading  # Importar threading

def open_file():
    # Abrir archivo de video
    file_path = filedialog.askopenfilename(filetypes=[("Archivos de video", "*.mp4;*.avi;*.mov")])
    if file_path:
        # Guardar la ruta del video para utilizarla en el futuro
        global video_path
        video_path = file_path

        # Capturar el primer fotograma del video
        cap = cv2.VideoCapture(video_path)
        ret, frame = cap.read()
        if ret:
            # Convertir el fotograma de BGR a RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Redimensionar el fotograma manteniendo la relación de aspecto
            max_width = 300
            max_height = 200
            h, w, _ = frame.shape
            aspect_ratio = w / h

            if w > max_width or h > max_height:
                if aspect_ratio > 1:
                    new_w = max_width
                    new_h = int(new_w / aspect_ratio)
                else:
                    new_h = max_height
                    new_w = int(new_h * aspect_ratio)
                frame_resized = cv2.resize(frame, (new_w, new_h))
            else:
                frame_resized = frame

            # Convertir a formato compatible con Tkinter
            img = Image.fromarray(frame_resized)
            img_tk = ImageTk.PhotoImage(img)

            # Actualizar la etiqueta con la imagen del primer fotograma
            label_img.config(image=img_tk)
            label_img.image = img_tk

            # Redimensionar la etiqueta para que coincida con las dimensiones de la imagen
            label_img.config(width=frame_resized.shape[1], height=frame_resized.shape[0])

        cap.release()

def generate_results():
    selected_option = combo_box.get()
    weight = entry_weight.get()
    match selected_option:
        case "Press banca":
            text = f"Seleccionado: {selected_option}, Peso: {weight}, {start_press(weight, video_path)}"
        case "Peso muerto":
            text = f"Seleccionado: {selected_option}, Peso: {weight}, {start_pesomuerto(weight, video_path)}"
        case "Sentadilla":
            text = f"Seleccionado: {selected_option}, Peso: {weight}, {start_sentadilla(weight, video_path)}"
        case _:
            text = "Elección no válida"
    results_text.set(text)
    adjust_label_width(text)
    
def adjust_label_width(text):
    """Ajusta dinámicamente el ancho del widget results_output según el texto."""
    max_length = max(len(line) for line in text.split("\n"))  # Longitud máxima por línea
    new_width = max(60, max_length)  # Evitar que sea más pequeño que el ancho inicial
    results_output.config(width=new_width)

def on_entry_click(event):
    if entry_weight.get() == "Ingrese peso:":
        entry_weight.delete(0, tk.END)
        entry_weight.config(foreground="black")

def on_focusout(event):
    if entry_weight.get() == "":
        entry_weight.insert(0, "Ingrese peso:")
        entry_weight.config(foreground="gray")

# Crear la ventana principal
root = tk.Tk()
root.title("Interfaz de ejemplo")

# Aplicar tema ttk
style = ttk.Style()
style.theme_use('clam')

# Personalizar widgets
style.configure('TButton', font=('Arial', 10), padding=5)
style.configure('TEntry', font=('Arial', 10))
style.configure('TLabel', font=('Arial', 10), padding=5)
style.configure('TCombobox', font=('Arial', 10))

# Crear widgets
button_open = ttk.Button(root, text="Abrir archivo", command=open_file)
label_img = tk.Label(root, text="Vista previa", background="lightgray", width=40, height=10)
combo_box = ttk.Combobox(root, values=["Press banca", "Peso muerto", "Sentadilla"], state='readonly')
entry_weight = ttk.Entry(root, foreground="gray")
entry_weight.insert(0, "Ingrese peso:")
entry_weight.bind("<FocusIn>", on_entry_click)
entry_weight.bind("<FocusOut>", on_focusout)
button_generate = ttk.Button(root, text="Generar", command=generate_results)
label_results = ttk.Label(root, text="Resultados:")
results_text = tk.StringVar()
results_output = ttk.Label(root, textvariable=results_text, anchor="w", background="white", width=60)
#video_path = ""
# Organizar widgets
button_open.grid(row=0, column=0, columnspan=3, padx=10, pady=10, sticky="ew")
label_img.grid(row=1, column=0, columnspan=3, padx=10, pady=10, sticky="nsew")
combo_box.grid(row=2, column=0, padx=10, pady=10, sticky="ew")
entry_weight.grid(row=2, column=1, padx=10, pady=10, sticky="ew")
button_generate.grid(row=2, column=2, padx=10, pady=10, sticky="ew")
label_results.grid(row=3, column=0, columnspan=3, padx=10, pady=5, sticky="w")
results_output.grid(row=4, column=0, columnspan=3, padx=10, pady=5, sticky="w")

# Ajustar columnas
root.columnconfigure(0, weight=1)
root.columnconfigure(1, weight=1)
root.columnconfigure(2, weight=1)

# Ejecutar la aplicación
root.mainloop()